In [9]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [10]:
data=pd.read_csv('/training.1600000.processed.noemoticon.csv', encoding='latin-1',  names=['target', 'ids', 'date', 'flag', 'user', 'text'])

In [11]:
data=data.drop(columns=['ids','date','flag','user'])
data['target']=data['target'].replace({0:0,4:1})
data['text'] = data['text'].str.replace(r'http\S+|www\S+|https\S+', '', regex=True)
data['text'] = data['text'].str.replace(r'@\w+', '', regex=True)
data['text'] = data['text'].str.replace(r'[^A-Za-z\s]', '', regex=True)
data['text']=data['text'].str.lower()

In [4]:
data.sample(100000)

In [12]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
texts = []
for text in data['text']:
    words = text.split()
    words = [ps.stem(word) for word in words if word not in stop_words]
    texts.append(' '.join(words))
data['text'] =  texts
tfidf = TfidfVectorizer(max_features=5000)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
x = tfidf.fit_transform(data['text']).toarray()
y = data['target']
X_train, X_test, y_train, y_test = train_test_split(x , y , test_size=0.2, random_state=42)

In [8]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print( classification_report(y_test, y_pred))

Accuracy: 0.76535
              precision    recall  f1-score   support

           0       0.78      0.74      0.76     10008
           1       0.75      0.79      0.77      9992

    accuracy                           0.77     20000
   macro avg       0.77      0.77      0.77     20000
weighted avg       0.77      0.77      0.77     20000

